## Loading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive

/content/drive


In [ ]:
%cd "My Drive"

/content/drive/My Drive


In [ ]:
%cd nlp

/content/drive/My Drive/nlp


In [ ]:
import pandas as pd

transformed_df = pd.read_csv("/content/drive/My Drive/nlp/data.csv")

## Split the document into smaller pieces

#### Smaller chunks of text

In [ ]:
tokenize = lambda s: len(nltk.word_tokenize(s))

average_words_mo = transformed_df[transformed_df['Type']=="Minneoppgave"]["Text"]
word_counts = average_words_mo.apply(tokenize).tolist()
average_word_count = sum(word_counts) / len(word_counts)
print(f'Minneoppgaver average words: {average_word_count}')

average_words_es = transformed_df[transformed_df['Type']=="Eventyr/Sagn"]["Text"]
word_counts = average_words_es.apply(tokenize).tolist()
average_word_count = sum(word_counts) / len(word_counts)
print(f'Eventyr/Sagn average words: {average_word_count}')

NameError: ignored

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
para_df = transformed_df.copy()

# define the chunk size
chunk_size = 50

# define a lambda function to split text into 50-word chunks
split_into_chunks = lambda text: [' '.join(nltk.word_tokenize(text[i:i+chunk_size]))
                                  for i in range(0, len(text), chunk_size)]

para_df['Text'] = para_df['Text'].apply(split_into_chunks)

In [ ]:
para_df.explode('Text')

,Concatenated,Text,Place,Date,Type
0,"Hordaland, Ulvik Ein gut vilde syna at det i...",Ein gut vilde syna at det ikkje beid hulderfolk .,"Hordaland, Ulvik",,Eventyr/Sagn
0,"Hordaland, Ulvik Ein gut vilde syna at det i...",Etter bufora drog han te fjells og la seg i ei...,"Hordaland, Ulvik",,Eventyr/Sagn
0,"Hordaland, Ulvik Ein gut vilde syna at det i...",l. Um ei rid kom det ein skokk med hulderfolk ...,"Hordaland, Ulvik",,Eventyr/Sagn
0,"Hordaland, Ulvik Ein gut vilde syna at det i...",ok te spela og dansa . Ei gjenta var burti gut...,"Hordaland, Ulvik",,Eventyr/Sagn
0,"Hordaland, Ulvik Ein gut vilde syna at det i...",lde pirast med honom . Drog han då kniven og h...,"Hordaland, Ulvik",,Eventyr/Sagn
...,...,...,...,...,...
2066,"Troms, Dyrøy Det var ein gong ein mann som sk...",ut og det galdt å koma seg på det tørre att . ...,"Troms, Dyrøy",,Eventyr/Sagn
2066,"Troms, Dyrøy Det var ein gong ein mann som sk...",yrja godsnakka og sistpå laut han ut med korfo...,"Troms, Dyrøy",,Eventyr/Sagn
2066,"Troms, Dyrøy Det var ein gong ein mann som sk...",n hadde fare slik fram . Og dei vart vener . N...,"Troms, Dyrøy",,Eventyr/Sagn
2066,"Troms, Dyrøy Det var ein gong ein mann som sk...","de han til ho , og han fekk ja . Og dei gifta ...","Troms, Dyrøy",,Eventyr/Sagn


### Splitting every document into sentences.

In [ ]:
!pip install nltk

import nltk

nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
tokenize = lambda s: nltk.sent_tokenize(s)

sentences = transformed_df.copy()

sentences['Text'] = sentences['Text'].apply(tokenize)

# Reset the index of the DataFrame
sentences = sentences.reset_index(drop=True)

In [ ]:
sentences['Text'][0]

[' Ein gut vilde syna at det ikkje beid hulderfolk',
 'Etter bufora drog han te fjells og la seg i eit sel',
 'Um ei rid kom det ein skokk med hulderfolk og tok te spela og dansa',
 'Ei gjenta var burti guten vilde pirast med honom',
 'Drog han då kniven og hogg te ne',
 'Då stogga leiken og alle kringa seg um guten og skreik Ha du merkt hinne so fær du ha ne',
 'Guten sprang te dørar og heim men då han nådde tunet var gjenta der og',
 'So totte han at han laut gifta seg med henne',
 'Fyrst presten nemnde gudsordi datt rumpa tå og dei levde sidan lenge og lukkeleg i hop']

In [ ]:
import string

sentences['Text'] = sentences['Text'].apply(lambda x: [s.translate(str.maketrans("", "", string.punctuation)) for s in x])

In [ ]:
# create a list of subsequent sentences for each row
subsequent_sentences = []
for row in sentences["Text"]:
    subsequent_sentences.extend([[row[i], row[i+1]] for i in range(len(row)-1)])

In [ ]:
import random

sentences_list = sentences['Text'].tolist()

random.shuffle(sentences_list)

num_pairs = 300000

nonsimilar_pairs = []

pairs = []
for i in range(num_pairs):
    row_1 = random.choice(sentences_list)
    
    available_rows = [row for row in sentences_list if row != row_1]
    row_2 = random.choice(available_rows)
    
    sentence_1 = random.choice(row_1)
    sentence_2 = random.choice(row_2)
    
    nonsimilar_pairs.append((sentence_1, sentence_2))

In [ ]:
nonsimilar_pairs[4]

('Det er slutt på køyrevegen gongevegen herfrå til sjøen er berre ein smal sti som går siksak ingen kan koma fram med kiste her',
 'Kona mi er død for 7 år sidan eg er nå i mitt 84 årPolitiske forhold som vart drøfta i min barndomsheim minnes eg lite til')

In [ ]:
ss = subsequent_sentences[10000:10100]

ns = nonsimilar_pairs[10000:10100]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel, AdamW
import torch

# Load pre-trained SBERT model and tokenizer
from transformers import AutoModel

tokenizer = AutoTokenizer.from_pretrained("NbAiLab/nb-bert-base")
model = AutoModel.from_pretrained("NbAiLab/nb-bert-base")

Some weights of the model checkpoint at NbAiLab/nb-bert-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=0aad46f047f768cfb217082abcd77e2867e0db0ee7c3d0bfac443deb7e662f60
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
data = ss+ns

In [ ]:
single_sentences = []
for pair in data:
    for entry in pair:
        single_sentences.append(entry.strip())

In [ ]:
# Apply the function to each flattened entry
encodings = tokenizer(single_sentences, truncation=True, padding=True, return_tensors='pt')

In [ ]:
encodings.convert_to_tensors()["input_ids"]

tensor([[  101, 31826, 10142,  ...,     0,     0,     0],
        [  101, 47199, 71393,  ...,     0,     0,     0],
        [  101, 47199, 71393,  ...,     0,     0,     0],
        ...,
        [  101,   142, 10240,  ...,     0,     0,     0],
        [  101,   208, 20220,  ...,     0,     0,     0],
        [  101, 10666, 10930,  ...,     0,     0,     0]])

In [ ]:
# Group the sentences back into their original sublists
new_list = []
for i in range(0, 399, 2):
    new_list.append([encodings[i], encodings[i+1]])

In [ ]:
ss = new_list[0:99]

ns = new_list[100:199]

In [ ]:
ss[0]

[Encoding(num_tokens=386, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=386, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [ ]:
from sentence_transformers import InputExample
train_examples = []
for text in ss:
  pair = [text[0], text[1]]
  train_examples.append({'texts': pair, 'label': 1})


for text in ns:
  pair = [text[0], text[1]]
  train_examples.append({'texts': pair, 'label': 0})

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class SamlaDataset(Dataset):
    def __init__(self, sentences):
      self.sentences = sentences
        
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentences = self.sentences[idx]
        input_seq_1, input_seq_2 = sentences['texts']
        input_id1 = input_seq_1
        input_id2 = input_seq_2
        label = torch.tensor([sentences['label']])
        return input_id1, input_id2, label

train_dataset = SamlaDataset(train_examples)
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# https://huggingface.co/transformers/v3.4.0/custom_datasets.html
from tqdm import tqdm

# Define loss function and optimizer
criterion = torch.nn.CosineSimilarity()
optimizer = AdamW(model.parameters(), lr=2e-5)

# Move model and tensors to GPU
device = torch.device("cuda")
model = model.to(device)

# Train SBERT model
for epoch in range(10):
    epoch_loss = 0

    with tqdm(total=len(dataloader.dataset)) as pbar:
      for input_id1, input_id2, labels in dataloader.dataset:
          optimizer.zero_grad()

          t1 = torch.tensor([input_id1.ids])
          t2 = torch.tensor([input_id2.ids])
          
          t1, t2, labels = (t1.to(device), t2.to(device), labels.to(device))

          output1 = model(t1).last_hidden_state.sum(dim=1) 
          output2 = model(t2).last_hidden_state.sum(dim=1) 

          similarity_score = criterion(output1, output2)
          loss = (1 - similarity_score) * labels + similarity_score * (1 - labels)
          loss = torch.mean(loss)
          loss.backward()
          optimizer.step()
          epoch_loss += loss.item()
          
          pbar.update(1)

    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(train_examples)}")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 198/198 [00:42<00:00,  4.67it/s]


Epoch 1 Loss: 0.4949409202484395


100%|██████████| 198/198 [00:42<00:00,  4.69it/s]


Epoch 2 Loss: 0.4924700984148064


100%|██████████| 198/198 [00:42<00:00,  4.69it/s]


...

Epoch 8 Loss: 0.4902181691593594


100%|██████████| 198/198 [00:42<00:00,  4.69it/s]


Epoch 9 Loss: 0.49005881885085445


100%|██████████| 198/198 [00:42<00:00,  4.68it/s]

Epoch 10 Loss: 0.487615380624328
